In [130]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys
import copy

from datetime import datetime

import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, Distance

from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier

import matplotlib.pyplot as plt
from matplotlib import colors

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Where is Proxima Centauri now

In [119]:
proxcen_ids = Simbad.query_objectids('Proxima Centauri')

proxcen_gaia_id = proxcen_ids[
    np.asarray(['Gaia' in name[0] for name in proxcen_ids])][0][0]

job = Gaia.launch_job(
    "select top 1 * from gaiadr2.gaia_source where source_id = {:s}".format(
        proxcen_gaia_id.split()[2]))

In [120]:
proxcen_gaia_dr2 = job.get_results()

# Proper motion from Gaia DR2
pm_ra_proxcen = u.Quantity(proxcen_gaia_dr2['pmra'])[0]
pm_dec_proxcen = u.Quantity(proxcen_gaia_dr2['pmdec'])[0]

# Distance in unit of pc from Gaia DR2 parallax
dist_proxcen = Distance(parallax=proxcen_gaia_dr2['parallax'][0] * u.mas)

#### Starting from J2000.0 (Simbad)

In [139]:
# Simbad record to get J2000.0 RA, Dec
proxcen_simbad = Simbad.query_object('Proxima Centauri')

j2000_year = Time('2000.0', format='decimalyear')

proxcen_coord_j2000 = SkyCoord(
    proxcen_simbad['RA'][0] + ' ' + proxcen_simbad['DEC'][0], unit=(u.hourangle, u.deg), 
    distance=dist_proxcen, pm_ra_cosdec=pm_ra_proxcen, pm_dec=pm_dec_proxcen,
    frame='icrs', obstime=j2000_year)

proxcen_coord_now = proxcen_coord_j2000.apply_space_motion(
    new_obstime=Time(datetime.utcnow()))

print(proxcen_coord_j2000)
print(proxcen_coord_now)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (217.42893792, -62.67949167, 1.30123556)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (-3781.30594097, 769.76565335, -5.38929096e-16)>
<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (217.38246694, -62.67514144, 1.30123563)
 (pm_ra, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (-8236.29858075, 772.49009358, 0.00904397)>


#### Starting from J2015.5 (Gaia DR2)

In [145]:
j2015 = Time(proxcen_gaia_dr2['ref_epoch'][0], format='decimalyear')

proxcen_coord_j2015 = SkyCoord(
    ra=u.Quantity(proxcen_gaia_dr2['ra'])[0],
    dec=u.Quantity(proxcen_gaia_dr2['dec'])[0],
    distance=dist_proxcen, 
    pm_ra_cosdec=pm_ra_proxcen, pm_dec=pm_dec_proxcen,
    obstime=j2015)
    
proxcen_coord_now = proxcen_coord_j2015.apply_space_motion(
    new_obstime=Time(datetime.utcnow()))

print(proxcen_coord_j2015)
print(proxcen_coord_now)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (217.39346574, -62.6761821, 1.30123556)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (-3781.30594097, 769.76565335, 0.)>
<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (217.38246259, -62.67515347, 1.30123556)
 (pm_ra, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (-8237.22724291, 770.4107708, 0.00308636)>
